In [7]:
#%pip install vaderSentiment
#%pip install geopandas
#%pip install textblob
%pip install Transformers
%pip install scipy
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/15/23/5d44decce68370cfeef76943bca3f2fb5b0057b7ab39f9d3b1feb53d64b8/torch-2.2.1-cp311-none-macosx_10_9_x86_64.whl.metadata
Using cached torch-2.2.1-cp311-none-macosx_10_9_x86_64.whl (150.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [50]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import re
from datetime import date
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import csv
from itertools import chain
from nltk.corpus import stopwords
from textblob import TextBlob
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import ast
from textblob import TextBlob
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
from time import time
import torch
from datetime import datetime
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
from datetime import datetime

from functools import reduce
from itertools import chain
import ast
#nltk.download('all')


In [63]:
original_data=pd.read_csv('reviews.csv')
original_data.head()
print(original_data['comments'].iloc[13:17].to_markdown())

|    | comments                                                                                                                                                                                                                                                                                                                                                                            |
|---:|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 13 | Unfortunately we weren't able to chat too much with Ali or David, because we were so busy, and they seem lovely!                                                                                                                       

In [64]:
def removeTags():
    L=[]
    with open('reviews.csv','r') as f:
        reader=csv.reader(f)
        c=0
        for row in reader:
            X=clean(row[5])
            L.append([row[0],row[1],row[2],row[3],row[4],X])
    with open('reviews_clean.csv','w') as f2:
        writer=csv.writer(f2)
        for row in L:
            writer.writerow(row)   
def clean(s):
    try:
        X=re.sub('<br/>'," ",s)
        X=re.sub("\n","",X)
        return X.strip()
    except:
        print(s)
    return s.strip()

In [65]:
removeTags() #Remove the html tags that are present in the comments field

In [73]:
reviews=pd.read_csv('reviews_clean.csv').dropna(subset=['comments'])
print(reviews['comments'].iloc[13:17].to_markdown())

|    | comments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|---:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [74]:
class TextNormalization:
    def __init__(self,df,column):
        #self.stop_words=[item for item in stopwords.words('english') if item!='not']
        self.stop_words=stopwords.words('english')
        contraction_patterns = [(r'can\'t', 'cannot'),
                                (r'haven\'t', 'have not'),
                                (r'wasn\'t', 'was not'),
                                (r'isn\'t', 'is not'),
                                (r'didn\'t', 'did not'),
                                (r'hasn\'t', 'has not'),
                                (r'weren\'t', 'were not')]
        self.contraction_regexes = [(re.compile(p), replaced_text) for p, replaced_text in contraction_patterns]
        self.wntl=WordNetLemmatizer()
        self.tokenizer=RegexpTokenizer(r'\w+') #Punctuation remover
        self.df=df
        self.column=column
        self.applyAll()
    def removeStopWords(self,s):
        word_tokens=self.tokenizer.tokenize(s) #Remove Punctuation
        return [self.wntl.lemmatize(word) for word in word_tokens if word not in self.stop_words]

    def do_contraction_normalization(self, text):
        for contraction_regex, replaced_text in self.contraction_regexes:
            text = contraction_regex.sub(replaced_text, text)
        return text

    def normalize(self,text):
        cleaned_tokens=self.removeStopWords(self.do_contraction_normalization(text))
        return pd.Series([cleaned_tokens,len(cleaned_tokens)])
    def applyAll(self):
        self.df[["Tokens",'Corpus_Length']]=self.df[self.column].apply(self.normalize)
        return self.df
    def get(self):
        return self.df

In [75]:
reviews=TextNormalization(reviews,'comments').get()
reviews.iloc[13:17]

,listing_id,id,date,reviewer_id,reviewer_name,comments,Tokens,Corpus_Length
13,12899,68414,2010-07-24,170065,Hope,Unfortunately we weren't able to chat too much...,"[Unfortunately, able, chat, much, Ali, David, ...",50
14,12899,71418,2010-07-31,89041,Ariel,We couldn't have asked for much more from our ...,"[We, asked, much, host, Ali, David, welcoming,...",20
15,12899,74541,2010-08-06,174803,Marilyn,"This was a perfect space in a charming, conven...","[This, perfect, space, charming, convenient, n...",12
16,12899,75259,2010-08-07,189922,Kathy,"Very clean, quiet, friendly hosts. We enjoyed...","[Very, clean, quiet, friendly, host, We, enjoy...",18


In [83]:
class sentimentVader:
    def __init__(self,df,column):
        self.df=df
        self.column=column
        self.sentimentAnalyzer=SentimentIntensityAnalyzer()
        self.applyAll()
    def getSentimentVader(self,toks):
        s=" ".join(toks)
        #s=toks
        sent = self.sentimentAnalyzer.polarity_scores(s)
        return pd.Series([float(sent['compound']),float(sent['neg']),float(sent['pos']),float(sent['neu'])])
    def applyAll(self):
        self.df[['sentimentVader','negVader','posVader','neutralVader']]=self.df[self.column].apply(self.getSentimentVader)
    def get(self): 
        return self.df

In [85]:
reviews=sentimentVader(reviews,'Tokens').get()
reviews.iloc[13:17]

,listing_id,id,date,reviewer_id,reviewer_name,comments,Tokens,Corpus_Length,sentimentVader,posVader,negVader,neutralVader
13,12899,68414,2010-07-24,170065,Hope,Unfortunately we weren't able to chat too much...,"[Unfortunately, able, chat, much, Ali, David, ...",50,0.9790,0.392,0.034,0.574
14,12899,71418,2010-07-31,89041,Ariel,We couldn't have asked for much more from our ...,"[We, asked, much, host, Ali, David, welcoming,...",20,0.8689,0.403,0.000,0.597
15,12899,74541,2010-08-06,174803,Marilyn,"This was a perfect space in a charming, conven...","[This, perfect, space, charming, convenient, n...",12,0.9371,0.643,0.000,0.357
16,12899,75259,2010-08-07,189922,Kathy,"Very clean, quiet, friendly hosts. We enjoyed...","[Very, clean, quiet, friendly, host, We, enjoy...",18,0.9489,0.562,0.000,0.438


In [86]:
class contractionReplacer:
    def __init__(self,contraction_patterns):
        self.contraction_regexes=[(re.compile(p),replaced_text) for p, replaced_text in contraction_patterns]
        
    def do_contraction_normalization(self,text):
        for contraction_regex, replaced_text in self.contraction_regexes:
            text=contraction_regex.sub(replaced_text, text)
        return text


def PartsOfSpeech(s):
    contraction_patterns=[(r'can\'t','cannot'),
                     (r'haven\'t','have not'),
                     (r'wasn\'t','was not'),
                     (r'isn\'t','is not'),
                (r'didn\'t','did not'),
                         (r'hasn\'t','has not'),
                         (r'weren\'t','were not')]
    nouns = []
    adjectives = []
    adjective_nouns=[]
    comma = ""
    comma2 = ""
    contractions_replaced=contractionReplacer(contraction_patterns).do_contraction_normalization(s)
    pos = TextBlob(contractions_replaced).tags
    for i in range(len(pos)):
        if 'NN' in pos[i][1]:
            nouns.append(pos[i][0])
        if 'JJ' in pos[i][1]:
            adj=pos[i][0]
            if i>1:
                for item in pos[i-2:i]:
                    if item[0].lower()=='not':
                        adj=item[0]+" "+adj
                        break
            if i<len(pos)-1 and 'NN' in pos[i+1][1]:
                following=pos[i+1][0]
                adjective_nouns.append(adj+" "+following)

            
            adjectives.append(adj)
            


    return pd.Series([nouns, adjectives, adjective_nouns])




In [87]:
reviews[['nouns','adjectives','adjectives_with_nouns']]=reviews['comments'].apply(PartsOfSpeech)

In [46]:
class SentimentAnalysis:
    def __init__(self,df,column):
        self.MODEL=f"cardiffnlp/twitter-roberta-base-sentiment"
        self.tokenizer = AutoTokenizer.from_pretrained(self.MODEL)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.MODEL)
        self.column = column
        self.df=self.getAllSentiment(df)



    def getSentiment(self,S):
        encoded_text = self.tokenizer(S, return_tensors='pt', max_length=512, truncation=True)
        output = (self.model(**encoded_text))
        scores = output[0][0].detach().numpy()
        L = softmax(scores)
        L=[str(item)+',' for item in L]
        D = {'neg': L[0], 'neu': L[1], 'pos': L[2]}
        return "".join(L)

    def getAllSentiment(self,df):
        df['sentimentRoberta']=self.getSentiment(str(df[self.column]))
        return df
    def get(self):
        return self.df


After contraction replacement Hello i'm not very happy.  I was not able to eat

tokens ['Hello', 'i', 'm', 'not', 'very', 'happy', 'I', 'was', 'not', 'able', 'to', 'eat']

New tokens ['Hello', 'happy', 'I', 'able', 'eat']


In [ ]:
reviews=SentimentAnalysisRoberta(reviews,'comments').get()
reviews[['negRoberta', 'neutralRoberta','posRoberta','nothing']] = reviews['sentimentRoberta'].str.split(' ', expand=True)
reviews=reviews.drop(columns=['sentimentRoberta','nothing'])
print(reviews.head())

In [45]:
listings=pd.read_csv('listingsDetailed.csv')
print(listings.info())
print(listings['name'].head())

['not']


In [ ]:

def splitNameCol(s):
    cats=[item.strip() for item in s.split('·')]
    name=cats[0]
    
    rating,bedrooms,beds,baths="No Rating",None,None,None
    for item in cats[1:]:
        if 'room' in item:
            bedrooms=item.split(' ')[0]
        elif 'Studio' in item:
            bedrooms=0.5
        elif 'bed' in item and 'room' not in item:
            beds=item.split(' ')[0]
        elif 'bath' in item:
            baths=item.split(' ')[0]
        elif '★' in item:
            rating=item.split('★')[1]
   
    return pd.Series([name, rating,bedrooms, beds, baths])
                
def convertTypeFloat(x):
    try:
        x=float(x)
    except:
        x=np.nan
    return x

def convertPrice(x):
    try:
        x=float(x.split('$')[1])
    except:
        x=np.nan
    return x


In [ ]:
listings[['Name', 'Rating','Bedrooms','Beds','Bathrooms']] = listings['name'].apply(splitNameCol)
listings['price']= listings['price'].apply(convertPrice)
listings['Rating']= listings['Rating'].apply(convertTypeFloat)
listings['Bedrooms'] = listings['Bedrooms'].apply(convertTypeFloat)
listings['Bathrooms'] = listings['Bathrooms'].apply(convertTypeFloat)

In [ ]:
merged=pd.merge(reviews,listings,left_on='listing_id', right_on='id',how='inner')
merged = merged[['listing_id', 'host_id', 'reviewer_id', 'host_name','Name', 'comments', 'sentimentVader', 'posVader', 'negVader',
             'neutralVader', 'negRoberta', 'neutralRoberta', 'posRoberta', 'Rating', 'nouns','adjectives_with_nouns', 'adjectives_' 'neighbourhood',
             'number_of_reviews', 'price', 'accommodates', 'Bedrooms', 'Bathrooms','Beds', 'reviews_per_month',
             'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value',
             'date', 'property_type', 'room_type','id_x']]

merged = merged.rename(columns={'Rating': 'host_rating','date':'date_reviewed','id_x':'id'})
merged['date_reviewed']=pd.to_datetime(merged['date_reviewed']).dt.date
merged.to_csv('airbnbData.csv')
merged.to_pickle('airbnbData.pkl')

In [88]:
reviews_df=pd.read_pickle('airbnbData.pkl')
print(reviews_df.info())

reviews_df['nouns']=reviews_df['nouns'].apply(lambda x: ast.literal_eval(x))
reviews_df['adjectives']=reviews_df['adjectives'].apply(lambda x: ast.literal_eval(x))
print(reviews_df.info())

FileNotFoundError: [Errno 2] No such file or directory: 'airbnbData.pkl'